# Heating python example  


Setup basic variables:

Boolean heater
float temp requested
float temp actual
timer heater
int valve (0 to 10)



#### Pseudo code:
set temp_req


get temp_act

if temp_req > temp_act:
    heating = True

while heating = true:
    valve_open
    delay 10 sec
    heating = on
        start_heating_timer (10)
    
while heating timer end:
    heating off

close valve


# pip install pyqt5
# pip install pyqt5-tools

import time
import datetime

class Heater:
    def __init__(self, state, minimum_duration):
        self.state = state
        self.minimum_duration = minimum_duration
        self.timer = 0

    def turn_on(self):
        self.timer = 0
        self.state = True

class Valve:
    def __init__(self, value):
        self.value = value

class Room:
    def __init__(self, name, valve, temp_req, temp_act):
        self.name = name
        self.valve = valve
        self.temp_req = temp_req
        self.temp_act = temp_act

    def check_heating_needed(self):
        if self.temp_act <= self.temp_req:
            return True
        return False

    def heating(self):
        time.sleep(1)
        self.temp_act = round(self.temp_act + 0.2, 2)

def main():
    #init variables 
    heater = Heater(False, 10)
    testroom1 = Room('testroom1', Valve(0), 18.5, 17.5)
    testroom2 = Room('testroom2', Valve(5), 18.5, 17.5)
    testroom3 = Room('testroom2', Valve(0), 18.5, 17.5)
    all_rooms = [testroom1, testroom2, testroom3]

    def other_valves_closed(all_rooms, my_room):
        for r in all_rooms:
            if not r.name == my_room.name:
                if r.valve.value > 0:
                    return False
                else: return True

    #internal functions
    def printactuals(room, heater):
        print(f"Actuals {str(datetime.datetime.now())} : Room: {room.name}; Req: {room.temp_req}; Act: {room.temp_act}; Valve= {room.valve.value} Heater state= {heater.state}; Heater timer= {heater.timer}")
    #main loop (never ending)
    while 2 > 1:
        printactuals(testroom1, heater)
        time.sleep(1)
        if (testroom1.check_heating_needed()):
            if testroom1.valve.value == 0: 
                testroom1.valve.value = 10 #open valve
                time.sleep(1) # delay - open valve heater start
            if not heater.state: heater.turn_on()
            # simulate heating increase
            if heater.state: testroom1.heating()

        if not (testroom1.check_heating_needed()): 
            #heating needed finished - stop heater
            print(f"{testroom1.name} req temp reached")
            
            if other_valves_closed(all_rooms, testroom1): #all valves are closed (except me !!!)
                while heater.timer <= heater.minimum_duration: #wait until heater timer reached min of 10
                    heater.timer += 1
                heater.state = False
                time.sleep(3) #delay
                testroom1.valve.value = 0 # close valve
                printactuals(testroom1, heater)
                print("heater is off - loop ended")
                break
        
        
        
        
        
        heater.timer += 1
          
main()


In [13]:
# pip install pyqt5
# pip install pyqt5-tools

import time
import datetime

class Heater:
    def __init__(self, state, minimum_duration):
        self.state = state
        self.minimum_duration = minimum_duration
        self.timer = 0

    def turn_on(self):
        self.timer = 0
        self.state = True

class Valve:
    def __init__(self, value):
        self.value = value

class Room:
    def __init__(self, name, valve, temp_req, temp_act):
        self.name = name
        self.valve = valve
        self.temp_req = temp_req
        self.temp_act = temp_act

    def check_heating_needed(self):
        if self.temp_act <= self.temp_req:
            return True
        return False

    def heating(self):
        self.temp_act = round(self.temp_act + 0.2, 2)

    def other_valves_closed(self, rooms_list):
        all_closed_flag = False
        for r in rooms_list:
            if r.name == self.name: continue #skip self
            if r.valve.value > 0:
                all_closed_flag = False
            else: 
                all_closed_flag = True
                continue
        return all_closed_flag
    def poll(self, heater):
        if (self.check_heating_needed()):
            if self.valve.value == 0: 
                self.valve.value = 10 #open valve
                time.sleep(1) # delay - open valve heater start
                print(f"{self.name} valve opened")
            if not heater.state: 
                heater.turn_on()
                print(f"{self.name} has started the heater")
            # simulate heating increase
            if heater.state: self.heating()
            return True
        if not (self.check_heating_needed()): 
            #heating needed finished - stop heater
            print(f"{self.name} req temp reached")
            return False

    def close_valve(self):
        self.valve.value = 0
        print(f"{self.name} valve closed")

    def stop_heater(self, heater):
        while heater.timer <= heater.minimum_duration: #wait until heater timer reached min of 10
            heater.timer += 1
        heater.state = False
        time.sleep(3) #delay
        print("Wait for heater to be stopped")        
        self.close_valve #close valve
        print("heater is off - loop ended")

def main():
    #init variables 
    heater = Heater(False, 10)
    testroom1 = Room('testroom1', Valve(0), 18, 17.5)
    testroom2 = Room('testroom2', Valve(0), 16, 17.5)
    testroom3 = Room('testroom3', Valve(0), 19, 17.5)
    all_rooms = [testroom1, testroom2, testroom3]
    heater_to_stop_flag = False
    #internal functions
    def printactuals(rooms, heater):
        for room in rooms:
            now = datetime.datetime.now().strftime("%H:%M:%S")
            print(f"Actuals {now}: {room.name}: Req: {room.temp_req}; Act: {room.temp_act}; Valve={room.valve.value} Heater= {heater.state}; timer= {heater.timer}")
    #main loop (never ending)
    while 2 > 1:
        break_out_flag = False
        printactuals(all_rooms, heater)
        time.sleep(1)
        for room in all_rooms:
            if room.poll(heater): 
                heater_to_stop_flag = False
                continue
            else:
                if not room.other_valves_closed(all_rooms): 
                    room.close_valve()
                else:
                    heater_to_stop_flag = True
                    last_room_heating = room
                    print(f"heater turn off flag set - last room : {room.name} , heater timer = {heater.timer}")
        if heater_to_stop_flag and not heater.timer <= heater.minimum_duration:
            #stop heater - else continue
            last_room_heating.stop_heater(heater)
            break_out_flag = True
            break
        if break_out_flag: break
        heater.timer += 1
          
main()

Actuals 18:24:40: testroom1: Req: 18; Act: 17.5; Valve=0 Heater= False; timer= 0
Actuals 18:24:40: testroom2: Req: 16; Act: 17.5; Valve=0 Heater= False; timer= 0
Actuals 18:24:40: testroom3: Req: 19; Act: 17.5; Valve=0 Heater= False; timer= 0
testroom1 valve opened
testroom1 has started the heater
testroom2 req temp reached
heater turn off flag set - last room : testroom2 , heater timer = 0
testroom3 valve opened
Actuals 18:24:43: testroom1: Req: 18; Act: 17.7; Valve=10 Heater= True; timer= 1
Actuals 18:24:43: testroom2: Req: 16; Act: 17.5; Valve=0 Heater= True; timer= 1
Actuals 18:24:43: testroom3: Req: 19; Act: 17.7; Valve=10 Heater= True; timer= 1
testroom2 req temp reached
testroom2 valve closed
Actuals 18:24:44: testroom1: Req: 18; Act: 17.9; Valve=10 Heater= True; timer= 2
Actuals 18:24:44: testroom2: Req: 16; Act: 17.5; Valve=0 Heater= True; timer= 2
Actuals 18:24:44: testroom3: Req: 19; Act: 17.9; Valve=10 Heater= True; timer= 2
testroom2 req temp reached
testroom2 valve closed